<a href="https://colab.research.google.com/github/SelmaDM/DesignPattern/blob/master/%C3%89valuation%20d%E2%80%99un%20syst%C3%A8me%20de%20recherche%20d'information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Université Paul Sabatier

EMIND1G1 - Fondements de la recherche d'information

**TP 3**

Enseignant : José G. Moreno

2023

## TP 3. Évaluation d’un système de recherche d'information

L'évaluation est une étape complexe dans la recherche d'information. Une des conférences qui a  largement aidé à l'avancement en cette matière est la conférence TREC (http://trec.nist.gov/). Dans ce TP nous nous intéressons à l'utilisation d'une des ces outils pour l'évaluation des moteurs des recherche. 

Pour l'évaluation nous avons besoin d'un fichier qui contient la « vérité de terrain » ou « gold standard » qui est normalement appelé qrel. Ce fichier contient pour chaque requête les identifiants des documents pertinents et non-pertinents. Également, il est nécessaire d'avoir des fichiers des résultats du moteur de recherche à évaluer.

Dans ce TP, nous allons utiliser un seul fichier qrel et plusieurs fichiers des résultats (chaque fichier des résultats sera évalué).

En continuation avec le TP2, considérez la phrase « Thomas and Mario are strikers playing in Munich ». Elle sera transforme en 3 requêtes  « Thomas », « Mario » et « Munich ». Chaque requête aura de documents considérés comme correctes (relevants) et incorrectes (no relevants).  La recherche de documents sera faite par votre système de recherche d’information. Cependant, la fait de dire qu’un document est relevant est une étape manuelle. Nous allons considérer les documents suivants comme relevants pour chaque requête :

> **Thomas** and **Mario** are strikers playing in **Munich**
>
>Thomas <br>
>* http://simple.wikipedia.org/wiki/Thomas_Müller
>
>
>Mario <br>
>* http://simple.wikipedia.org/wiki/Mario_Gómez <br>
>* http://simple.wikipedia.org/wiki/Mario_Götze
>
>
>Munich <br>
>* http://simple.wikipedia.org/wiki/FC_Bayern_Munich

Maintenant, il suffit d’utiliser vos résultats de chaque requête dans le format TREC pour les évaluer. Pour simplicité, nous allons utiliser la librarie [pytrec_eval](https://github.com/cvangysel/pytrec_eval) qui est un wrapper du logiciel [trec_eval](http://trec.nist.gov/trec_eval/trec_eval.8.1.tar.gz)

Pour information, voici le fichier qrel pour les 3 requêtes précédentes :

```
101 0 Thomas_Müller 1
101 0 Thomas_Edison 0
101 0 Thomas_the_Apostle 0
102 0 Mario_Gómez 1
102 0 Mario_Götze 1
103 0 FC_Bayern_Munich 1
```

Notez que nous allons utiliser pytrec_eval, qui utilise un dictionaire pour le qrel au lieu d'un fichier.

Notez que la première colonne est l’identifiant de la requête (nous avons trois valeurs différents, une pour chaque requête), suivi de zéro (0), suivi de l’identifiant du document annoté (le titre de la page Wikipédia) et une valeur pour dire si le document est relevant (1) ou non (0). Notez aussi que les qrels contient des documents pertinents et des documents non-pertinents.

Puis il faut créer le fichier des résultats avec la sortie de votre programme fait pendant les Tps précédents. Pour information, voici un fichier résultat d’un système :

```
101	Q0	Thomas_Edison	1	  5.5	STANDARD
101	Q0	Thomas_Müller	2	  4.4	STANDARD
101	Q0	Thomas_the_Apostle	3	  3.3	STANDARD
101	Q0	Isiah_Thomas	4	  2.2	STANDARD
101	Q0	Thomas_Aquinas	5	  1.1	STANDARD
102	Q0	Mario	1	  5.5	STANDARD
102	Q0	Super_Mario	2	  4.4	STANDARD
102	Q0	Super_Mario_Bros.	3	  3.3	STANDARD
102	Q0	Super_Mario_Bros._2	4	  2.2	STANDARD
102	Q0	Mario_(series)	5	  1.1	STANDARD
102	Q0	Super_Mario_World	6	  1.0	STANDARD
102	Q0	Super_Mario_Bros._3	7	  0.9	STANDARD
102	Q0	New_Super_Mario_Bros.	8	  0.8	STANDARD
102	Q0	Mario_Gómez	9	  0.7	STANDARD
102	Q0	Mario_Party_4	10	  0.6	STANDARD
103	Q0	Munich	1	  5.5	STANDARD
103	Q0	FC_Bayern_Munich	2	  4.4	STANDARD
103	Q0	Munich_Airport	3	  3.3	STANDARD
103	Q0	Munich_Agreement	4	  2.2	STANDARD
103	Q0	Munich_Rural_District	5	  1.1	STANDARD
```

Notez, que comme pour les qrels, pytrec_eval utilise un dictionaire pour le résultat d’un système au lieu d'un fichier.

La première colonne est l’identifiant de la requête (la même que pour le qrel), suivi de zéro (Q0), suivi de l’identifiant du document retrouvé par votre système (le titre de la page Wikipédia), suivi de la position du documents dans les résultats, suivi de la valeur de similarité donné par le modèle de poids choisi et de l’identifiant du système (votre nom par exemple).

Une fois construit les fichiers qrels et résultats, nous pouvons utiliser le logiciel d'évaluation trec_eval pour obtenir les résultats de l'évaluation. Cependant, pour simplicité nous allons utiliser pytrec_eval. Donc, pour pytrec_eval, il suffit de déclarer les deux dictionaires (qrel et run) et en suite appeler la méthode ```relevanceEvaluator``` comme indiqué dans l'exemple ci-dessous. 

In [ ]:
!pip install pytrec_eval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp39-cp39-linux_x86_64.whl size=293184 sha256=14c78606f710ce502297a5ab2dfda234c5d241d986a83400cd52745ae360fa2a
  Stored in directory: /root/.cache/pip/wheels/e9/91/35/6059501bca98e27e0b4f91ecaaff86c95ca7f4919ff22f0d54
Successfully built pytrec_eval


Création des fichier examples

In [ ]:
import pytrec_eval
import json
import pandas as pd

In [ ]:
qrel = {
    '101': {
        'Thomas_Müller': 1,
        'Thomas_Edison': 0,
        'Thomas_the_Apostle': 0,
    },
    '102': {
        'Mario_Gómez': 1,
        'Mario_Götze': 1,
    },
    '103': {
        'FC_Bayern_Munich': 1,
    },
}


In [ ]:
run = {
    '101': {
        'Thomas_Edison': 5.5,
        'Thomas_Müller': 4.4,
        'Thomas_the_Apostle': 3.3,
        'Isiah_Thomas': 2.2,
        'Thomas_Aquinas': 1.1,
    },
    '102': {
        'Mario': 10.10,
        'Super_Mario': 9.9,
        'Super_Mario_Bros.': 8.8,
        'Super_Mario_Bros._2': 7.7,
        'Mario_(series)': 6.6,
        'Super_Mario_World': 5.5,
        'Super_Mario_Bros._3': 4.4,
        'New_Super_Mario_Bros.': 3.3,
        'Mario_Gómez': 2.2,
        'Mario_Party_4': 1.1,
    },
    '103': {
        'Munich': 5.5,
        'FC_Bayern_Munich': 4.4,
        'Munich_Airport': 3.3,
        'Munich_Agreement': 2.2,
        'Munich_Rural_District': 1.1,
    },
}


Évaluation de l'exemple

In [ ]:
evaluator = pytrec_eval.RelevanceEvaluator(
    qrel, {'map', 'ndcg'})

pd.DataFrame(evaluator.evaluate(run)).T

,map,ndcg
101,0.500000,0.630930
102,0.055556,0.184576
103,0.500000,0.630930


Chaque clé corresponde au résultat d’une métrique d’évaluation pour les trois requêtes. 

###1. Requêtes

Utilisez les suivants requêtes dans votre système et générez les résultats dans le format décrit précédemment (variable ```run```)  :

```
ID:100+i
Thomas and Mario are strikers playing in Munich

ID:200+i
Leo scored two goals and assisted Puyol to ensure a 4–0 quarter-final victory over Bayern

ID:300+i
Skype software for Mac

ID:400+i
Cowboys fans petition Obama to oust Jones

ID:500+i
Kate and Henry are known for being devoted to the Anglican church
```



In [ ]:
# 1. Requêtes 
#code pour executer mon programme du TP 02

!gdown --id 16lfBW5guTbIIE7Z7d-hnMJH0SrwzZlpd
!unzip pd_index.zip

# déclaration de la variable JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
!export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

#installation de pyterrier avec pip
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

#Initialization de JVM
import pyterrier as pt
if not pt.started():
  pt.init()


indexref2 = pt.autoclass("org.terrier.querying.IndexRef").of(os.path.join("/content/pd_index", "data.properties"))


/usr/local/lib/python3.9/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16lfBW5guTbIIE7Z7d-hnMJH0SrwzZlpd
To: /content/pd_index.zip
100% 16.4M/16.4M [00:01<00:00, 12.6MB/s]
Archive:  pd_index.zip
   creating: pd_index/
  inflating: pd_index/data.inverted.bf  
  inflating: pd_index/data.properties  
  inflating: pd_index/data.document.fsarrayfile  
  inflating: pd_index/data.direct.bf  
  inflating: pd_index/data.meta.idx  
  inflating: pd_index/data.lexicon.fsomapid  
  inflating: pd_index/data.lexicon.fsomapfile  
  inflating: pd_index/data.meta-0.fsomapfile  
  inflating: pd_index/data.meta.zdata  
  inflating: pd_index/data.lexicon.fsomaphash  
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/terrier-org/pyterrier

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [ ]:
# 1. Requêtes 
import json
query = "Thomas and Mario are strikers playing in Munich"
#query = "Leo scored two goals and assisted Puyol to ensure a 4–0 quarter-final victory over Bayern"
#query = "Skype software for Mac"
#query = "Cowboys fans petition Obama to oust Jones"
#query = "Kate and Henry are known for being devoted to the Anglican church"

termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
print("termToSearch : ", termToSearch ,"\n")
id_requete = [101,102,103]
for term,page,id in zip(termToSearch, 'term',id_requete):
    dfres = pt.BatchRetrieve(indexref2, wmodel="BM25",metadata=["docno","title","url"]).search(term)[['title','score']]
    index = dfres.index
    index.name = id #Add name to the dataframe
    #Convert dataframe to json format
    result = dfres.head(5).to_json(orient="index")
    parsed = json.loads(result)
    y = json.dumps(parsed, indent=4)  
    print(print(y))

termToSearch :  ['Thomas', 'Mario', 'Munich'] 

{
    "0": {
        "title": "Wikipedia: Tom Spurgeon",
        "score": 13.5349445342
    },
    "1": {
        "title": "Wikipedia: Gospel of Thomas",
        "score": 12.7161849708
    },
    "2": {
        "title": "Wikipedia: BSC Young Boys",
        "score": 12.5644027772
    },
    "3": {
        "title": "Wikipedia: M. T. Liggett",
        "score": 12.5644027772
    },
    "4": {
        "title": "Wikipedia: People's Party (United States)",
        "score": 12.5644027772
    }
}
None
{
    "0": {
        "title": "Wikipedia: Mario (disambiguation)",
        "score": 16.7689520345
    },
    "1": {
        "title": "Wikipedia: Super Mario Land 2: 6 Golden Coins",
        "score": 15.8338871696
    },
    "2": {
        "title": "Wikipedia: Club Deportivo Universidad Catlica",
        "score": 15.8338871696
    },
    "3": {
        "title": "Wikipedia: Super Mario",
        "score": 15.463807894
    },
    "4": {
        "title": 

2. Qrels

Utilisez le qrel ***qreltp*** déclaré ci-dessous

In [ ]:
qreltp = {
    '101': {
        'Thomas_Müller': 1,
        'Thomas_Edison': 0,
        'Thomas_the_Apostle': 0,
    },
    '102': {
        'Mario_Gómez': 1,
        'Mario_Götze': 1,
    },
    '103': {
        'FC_Bayern_Munich': 1,
    },
    '201': {
        'Lionel_Messi': 1,
    },
    '202': {
        'Carles_Puyol': 1,
    },
    '203': {
        'FC_Bayern_Munich': 1,
    },
    '301': {
        'Skype': 1,
    },
    '302': {
        'Mac_OS': 1,
    },
    '401': {
        'Dallas_Cowboys': 1,
    },
    '402': {
        'Barack_Obama': 1,
    },
    '403': {
        'Jerry_Jones': 1,
    },
    '501': {
        'Catherine_Duchess_of_Cambridge': 1,
    },
    '502': {
        'Prince_Harry': 1,
    },
    '503': {
        'Anglicanism': 1,
    },
}

### 3. Configurations
Générez au moins 5 configurations différents de votre système avec 100 résultats et évaluez-les. Comment expliquez-vous vos résultats ?

### 4. Résultats
Avec les mêmes 5 configurations, générez 1000 résultats et évaluez-les. Il y a-t-il des différences dans certains métriques ? Pourquoi ?

### 5. Analyses
Faites une comparaison entre les résultats des différents configurations. Quelles métriques ont changés ?

In [ ]:
# 2. Configuration  
import json
query = "Thomas and Mario are strikers playing in Munich"
#query = "Leo scored two goals and assisted Puyol to ensure a 4–0 quarter-final victory over Bayern"
#query = "Skype software for Mac"
#query = "Cowboys fans petition Obama to oust Jones"
#query = "Kate and Henry are known for being devoted to the Anglican church"

termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
print("termToSearch : ", termToSearch ,"\n")
id_requete = [101,102,103]
for term,page,id in zip(termToSearch, 'term',id_requete):
    dfres = pt.BatchRetrieve(indexref2, wmodel="BM25",metadata=["docno","title","url"]).search(term)[['title','score']]
    index = dfres.index
    index.name = id #Add name to the dataframe
    #Convert dataframe to json format
    result = dfres.head(100).to_json(orient="index") #100 résultats
    parsed = json.loads(result)
    y = json.dumps(parsed, indent=4)  
    print(print(y))


termToSearch :  ['Thomas', 'Mario', 'Munich'] 

{
    "0": {
        "title": "Wikipedia: Tom Spurgeon",
        "score": 13.5349445342
    },
    "1": {
        "title": "Wikipedia: Gospel of Thomas",
        "score": 12.7161849708
    },
    "2": {
        "title": "Wikipedia: BSC Young Boys",
        "score": 12.5644027772
    },
    "3": {
        "title": "Wikipedia: M. T. Liggett",
        "score": 12.5644027772
    },
    "4": {
        "title": "Wikipedia: People's Party (United States)",
        "score": 12.5644027772
    },
    "5": {
        "title": "Wikipedia: Thomas (name)",
        "score": 11.9908322685
    },
    "6": {
        "title": "Wikipedia: Thomas Tallis",
        "score": 11.8557800878
    },
    "7": {
        "title": "Wikipedia: Colleges and universities in the U.S. Virgin Islands",
        "score": 11.8557800878
    },
    "8": {
        "title": "Wikipedia: Frank-Walter Steinmeier",
        "score": 11.8557800878
    },
    "9": {
        "title": "Wikipe